# Welcome to Augmentoolkit

This notebook is where you generate all your data.

Augmentoolkit is meant to allow instruct-tuned models to learn from books, even using themselves to generate new data through a sort-of bootstrapping method. It is meant to stop model creators from having to work as data annotators, and not actual model trainers. It is meant to allow anyone to make their own high-quality dataset with thousands of entries.

## Quickstart:

- Get this notebook and the other repo code onto a machine with the power to run Airoboros-l2-70b-3.1.2.Q4_K_M
- Put the model into ./logical_models (relative to this notebook). 
- Run all the cells below and watch as the notebook generates questions, answers, and conversations based on Principles of Chemistry, Simple Sabotage, and Introduction to Philosophy.

If you want to add your own texts, follow the instructions in list item #1 above.

### Note: this notebook makes roughly 1/3 characters generated to be **mildly NSFW** by default. You will need to modify the character personality code in `./generation_functions/special_instructions.py` or use "Assistant mode" if you want something cleaner.

## Customization:
### Here are some ways you can adapt this notebook to your use case, along with a brief description of how to do so, arranged in increasing order of difficulty (this information is also available in the README):
1. ***Change the source texts used to generate training data.*** You can do this in the cell right below this one. **IMPORTANT** the filenames of these should be formatted in a specific way, since the filenames are used as part of the prompts and in at least one regex. You need to have them be like: `[textname], by authorname`. You can also include the publication date after the author name if you want, but note that this will tend to bias most of the characters to live in the era of the textbook, which may or may not be what you want.

2. ***Change the personalities of the characters generated.*** Currently, when generating characters for the multiturn conversation step, three randomly-selected traits are appended to the "special instructions" set of the prompt to constrain what kind of character is generated by the model. Depending on what kind of model you want to make, or even just if your preferences vary, then you will probably want to modify this a bit. You can do so in `./generation_functions/special_instructions.py`. A more in-depth description of the trait-axis system that I (over)thought up is available in the comments of that file.

3. ***Change the constants.*** There are a few constant values in this notebook, and in `./generation_functions/constant_values.py`. These constants are tested, but if your use case requires special settings (e.g., you want to make conversations from more permutations of existing questions; or you think the character counts for the "duplicate question/answer" validation functions are too restrictive) then feel free to change the related setting. The most intuitive and least-likely-to-break-anything settings to change are rearrangements_to_take and double_check_counter. Beyond that... you'll need to figure out what the function does before changing it if you expect it to run.

4. ***Assistant Mode*** Technically this could be considered part of 3), but it's different enough that I feel it warrants separate explanation. By default, the notebook is configured to produce RP-style data; "Assistant mode" is something you can toggle in the settings cell immediately below this one, which skips character and scenario generation and answers every question in a chat between a user and a helpful AI assistant (with no personality). In the limited testing I have done with this, **it seems that assistant mode is simple enough to work with 13b models** such as Flatorcamaid by Ikari. So if your compute or time are very limited, or you are using this for a more professional use case, feel free to turn this on.

5. ***Change the model.*** This is as simple as switching the LOGICAL_MODEL parameter out for another one, but your mileage may vary, significantly. You will also have to adjust RoPE scaling for non-llama 2 models -- e.g., if you're using Mixtral, don't leave `rope_freq_scale=0.33`, which 3xes the context (you do not need 96k context, only 12k).

6. ***Change the examples.*** If you change the examples you can completely overhaul what this notebook does, but this requires a lot of prompting skill and possibly huge amounts of time to get it working again (source: most of my last three months were spent prompting, and most of this prompting was spent on the examples). Unless you want to convert this notebook from question-and-answer generation to some completely other task, I'd recommend changing only the conversation generation prompts -- they're a bit less finnicky, and if you just want to change the kind of characters generated (maybe you want a different writing style) that's where you'd find the differences.



In [ ]:
# NOTE NOTEBOOK SETTINGS AND CONSTANTS (some script file constants are in generation_functions/constants.py)

# Put your desired quant of your desired model in the relevant directories


# "airoboros-l2-70b-3.1.2.Q4_K_M.gguf" <- recommended for the large logical model
# "flatorcamaid-13b-v0.2.Q8_0.gguf" <- recommended for the normal logical model
# A6000s on Vast.ai are a good choice for running this notebook

LOGICAL_MODEL = "TheBloke/OpenHermes-2.5-Mistral-7B-GPTQ"  # model used for decision-making and base question generation (should be "smart")
LARGE_LOGICAL_MODEL = "TheBloke/OpenHermes-2.5-Mistral-7B-GPTQ"
ASSISTANT_MODE = False  # change to true if you want all conversations to be with an "AI language model" and not characters. Useful for more professional use cases.
DOUBLE_CHECK_COUNTER = 3  # Set to 1 to check outputs only once; set to 2 to check twice; set to 3 to check thrice, etc. Set to 0 to break everything in vet_question_loop() and elsewhere. Set to -1 and cause the universe to implode?

REARRANGEMENTS_TO_TAKE = 3  # How many of the possible permutations of tuples in a group to take and make multiturn convs out of. Adjust higher to get more data out of less text, but it might be a bit repetitive. NOTE your eval loss will be basically worthless if you aren't careful with how you shuffle your dataset when you're about to train.

source_texts = [
    "Simple Sabotage, by the Office of Strategic Services, published 1944.txt",
    "Principles of Chemistry, by Demitry Mendeleev, published 1897.txt",
]

### Start running the Aphrodite server (additional requirements: requests and aphrodite)

## Below: Defines and imports functions that you will probably use no matter what cells in the notebook you choose to run:

In [2]:
import os
import uuid

multi_turn_convs_info_dir = "./multi_turn_convs_info"  # we generate all the information fed to the multiturn prompt, and generate the actual multiturn prompt, separately; since every step but the last is capable of being done by a 13b

def make_id():
    return str(uuid.uuid4())


def write_output_to_file(output, directory, uuid):
    # Ensure directory exists
    if not os.path.exists(directory):
        os.makedirs(directory)

    # Define the file path using the directory and UUID
    file_path = os.path.join(directory, f"{uuid}.txt")

    # Write the output to the file
    with open(file_path, "w") as file:
        file.write(output)

    print(f"Output written to {file_path}")

# This is in no way best practices, but all my prompts being searchable and separate files is a good way to make my life easier.
import pkgutil
import importlib
import sys

sys.path.append("./generation_functions")
sys.path.append("./control_flow_functions")

import generation_functions  # This is the package directory
from control_flow_functions import control_flow_functions

# First, import all modules so they can be reloaded
for _, module_name, _ in pkgutil.iter_modules(
    generation_functions.__path__, generation_functions.__name__ + "."
):
    importlib.import_module(module_name)

# Now, reload each module and import all callable attributes
for _, module_name, _ in pkgutil.iter_modules(
    generation_functions.__path__, generation_functions.__name__ + "."
):
    # Reload the module
    module = importlib.reload(sys.modules[module_name])
    # Iterate through each attribute in the reloaded module
    for attribute_name in dir(module):
        # Retrieve the attribute
        attribute = getattr(module, attribute_name)
        if callable(attribute):
            # If it's callable, it's a function or class, so you set it in the globals dictionary
            globals()[attribute_name] = attribute



# Start Aphrodite Engine
engine_wrapper = EngineWrapper(model=LOGICAL_MODEL, quantization="gptq")

ModuleNotFoundError: No module named 'aphrodite'

In [ ]:
from transformers import AutoTokenizer
import re
from tqdm import tqdm
import nltk

nltk.download("punkt")
from nltk.tokenize import sent_tokenize

tokenizer = AutoTokenizer.from_pretrained(
    "Gryphe/MythoMax-L2-13b"
)  # It doesn't matter what model goes here as long as it is sentencepiece

sentence_chunks = []
for source_text in source_texts:
    sentence_chunks += control_flow_functions.sentence_chunking_algorithm(source_text, tokenizer)

conversions = [("\n", " "), ("  ", " ")]

paragraphs_processed = [
    (control_flow_functions.fix_text(conversions, seq[0]), seq[1]) for seq in sentence_chunks
]

#### Inspect various features of the text you have fed in to see if it came out alright-ish

In [ ]:
len(paragraphs_processed)

In [ ]:
paragraphs_processed[5]

#### The below cell will take a while to start generating, because it only uses one thread due to Python's global interpreter lock. So what it does it it schedules every single filtering first (that's what the active thread does) then the active thread jumps between all the calls that schedule the llm execution.

In [3]:
import json
import os
from tqdm import tqdm
import asyncio
from generation_functions import judge_paragraph

# Create directory if it doesn't exist
output_dir = "./worthy_for_questions"
os.makedirs(output_dir, exist_ok=True)

# Determine which paragraphs are worthy of making questions from
judged_worthy_for_questions = []

await control_flow_functions.filter_all_questions(paragraphs_processed, judged_worthy_for_questions, engine_wrapper, output_dir, judge_paragraph.judge_paragraph)

NameError: name 'filter_all_questions' is not defined

In [ ]:
filtered_worthy_for_questions = control_flow_functions.filter_and_graph(judged_worthy_for_questions)

In [ ]:
print(filtered_worthy_for_questions[0])

In [ ]:
# Control flow helpers

# Change this value to change how many times the checks must pass consecutively for a thing to be accepted

import logging
from math import ceil
import traceback


# Setup logging
# Except I actually don't use this because I switched to print() because jupyter is annoying with logging
logging.basicConfig(
    filename="data_generation.log",
    filemode="a",
    format="%(asctime)s - %(levelname)s - %(message)s",
    level=logging.INFO,
)


async def vet_answer_accuracy_loop(qa_tuple, total_retries, run_id):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting ACCURACY loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        passed_checks = 0
        times_checked = 0
        dissenting_reasoning = ""
        while times_checked < DOUBLE_CHECK_COUNTER:
            print(
                f"\n\nACCURACY CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
            )
            judgement, answer_accuracy_output = await check_answer(qtuple, engine_wrapper)
            write_output_to_file(
                answer_accuracy_output, "./check_answer_accuracy_generations", run_id
            )
            if not judgement[0]:  # if not accurate
                dissenting_reasoning = judgement[1]
            else:
                passed_checks += 1
            times_checked += 1
            if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break
            failed_checks = times_checked - passed_checks
            if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break

        if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):  # if question checks passed
            print(f"\n\ANSWER ACCURACY CHECKS PASSED retries: {total_retries}")
            return qtuple
        else:
            # Generate new question and restart the loop
            print(
                f"\n\nACCURACY CHECKS FAILED - SENDING BACK TO QUESTION LOOP retries: {total_retries}"
            )
            total_retries += 1
            qtuple, generate_new_q_output = await generate_new_question(qtuple, engine_wrapper)
            write_output_to_file(
                generate_new_q_output, "./regenerate_question_generations", run_id
            )
            vet_question_loop(
                qtuple, total_retries, question_group_id=run_id.split("--subquestion--")[0]
            )  # going to get one hell of a call stack by the end of this, but it should be fine
    except Exception as e:
        print("!!ERROR!!")
        print(e)
        traceback.print_exc()

    return (None, None, None, qtuple[3])


async def vet_answer_relevance_loop(qa_tuple, total_retries, run_id):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting RELEVANCE loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        passed_checks = 0
        times_checked = 0
        dissenting_reasoning = ""
        while times_checked < DOUBLE_CHECK_COUNTER:
            print(
                f"\n\nRELEVANCE CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
            )
            judgement, answer_relevancy_output = await check_answer_relevancy_with_text(
                qtuple, engine_wrapper
            )
            write_output_to_file(
                answer_relevancy_output, "./check_answer_relevancy_generations", run_id
            )
            if not judgement[0]:  # if not relevant
                dissenting_reasoning = judgement[1]
            else:
                passed_checks += 1
            times_checked += 1
            if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break
            failed_checks = times_checked - passed_checks
            if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                break

        if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
            print(f"\n\nRELEVANCE CHECKS PASSED")
            return await vet_answer_accuracy_loop(qtuple, total_retries, run_id)
        else:
            print(f"\n\nRELEVANCE CHECKS FAILED - SENDING BACK TO QUESTION LOOP")
            total_retries += 1
            qtuple, generate_new_q_output = await generate_new_question(qtuple, engine_wrapper)
            write_output_to_file(
                generate_new_q_output, "./regenerate_question_generations", run_id
            )
            return vet_question_loop(
                qtuple, total_retries, question_group_id=run_id.split("--subquestion--")[0]
            )
    except Exception as e:
        print("!!ERROR!!")
        print(e)
        traceback.print_exc()

    return (None, None, None, qtuple[3])


async def vet_question_loop(qa_tuple, question_group_id=None, total_retries=0):
    try:
        qtuple = qa_tuple
        print(
            f"\n\nStarting QUESTION loop for question: {qtuple[0]}, context: {qtuple[2]}"
        )
        while total_retries <= 4:
            run_id = question_group_id + "--subquestion--" + make_id()
            passed_checks = 0
            times_checked = 0
            dissenting_reasoning = ""
            while times_checked < DOUBLE_CHECK_COUNTER:
                print(
                    f"\n\nQUESTION CALL CHECK ANSWER: {qtuple[0]}, context: {qtuple[2]}, retries: {total_retries}, dissenting reasoning: {dissenting_reasoning}"
                )
                judgement, check_q_output = await check_question(qtuple, engine_wrapper)
                write_output_to_file(
                    check_q_output, "./check_question_generations", run_id
                )
                if not judgement[0]:  # if not relevant
                    dissenting_reasoning = judgement[1]
                else:
                    passed_checks += 1
                times_checked += 1
                if passed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                    break
                failed_checks = times_checked - passed_checks
                if failed_checks >= ceil(DOUBLE_CHECK_COUNTER / 2):
                    break

            if passed_checks >= ceil(
                DOUBLE_CHECK_COUNTER / 2
            ):  # if all question checks passed
                print(f"\n\nQUESTION CHECKS PASSED retries: {total_retries}")
                return await vet_answer_relevance_loop(qtuple, total_retries, run_id)
            else:
                # Generate new question and restart the loop
                print(
                    f"\n\nQUESTION CHECKS FAILED - GENERATING NEW QUESTION retries: {total_retries}"
                )
                total_retries += 1
                if (
                    total_retries <= 4
                ):  # only regen question if we're not already at max regens
                    qtuple, generate_new_q_output = await generate_new_question(
                        qtuple, engine_wrapper
                    )
                    write_output_to_file(
                        generate_new_q_output,
                        "./regenerate_question_generations",
                        run_id,
                    )
                    print("New question: ", qtuple)
                # no calling of vet_question_loop, since we're already in a while loop
    except Exception as e:
        print("!!ERROR!!")
        print(e)
        traceback.print_exc()

    return (None, None, None, qtuple[3])

In [ ]:
# control flow
import json
import os
import glob

# Directory for QA tuples
qa_tuples_dir = "./qatuples_raw"
if not os.path.exists(qa_tuples_dir):
    os.makedirs(qa_tuples_dir)

# Initialize vetted_qa_tuples
vetted_qa_tuples = []  # tuple list of qa tuples that have been judged good

async def generate_qatuples_from_para(idx,para):
    try:
        existing_files = glob.glob(
            os.path.join(qa_tuples_dir, f"para_{idx}_*.json")
        )  # check if qs already exist

        if len(existing_files) > 0:  # If files exist, skip this paragraph entirely
            print(f"Skipping para_{idx} as files already exist; loading said files")
            for file_path in existing_files:
                with open(file_path, "r") as file:
                    qa_tuple = tuple(json.load(file))
                vetted_qa_tuples.append(qa_tuple)
                continue

        question_group_id = make_id()
        print(f"\n\n\nOUTER LOOP CALL GENERATE QPLAN para: {para}, \n\n idx: {idx}")
        plan, questions_plan_output = await generate_questions_plan(para, engine_wrapper)
        write_output_to_file(
            questions_plan_output, "./question_plan_generations", question_group_id
        )
        print(
            f"\n\n\nOUTER LOOP CALL GENERATE Q: {para}, \n\n idx: {idx} \n\n plan: {plan}"
        )
        question_answer_tuples, question_generation_output = await generate_questions(
            para, plan, engine_wrapper
        )
        write_output_to_file(
            question_generation_output,
            "./question_generation_generations",
            question_group_id,
        )
        for qnum, question_answer_tuple in enumerate(question_answer_tuples):
            print(
                f"\n\n=======!!=BEGIN VETTING QA TUPLE {idx}_{qnum}=!!=======\n\n"
            )
            good_qa_tuple = await vet_question_loop(
                question_answer_tuple, question_group_id=question_group_id
            )

            # Write resulting question file if the tuple is not None
            if good_qa_tuple[0] is not None:
                file_path = os.path.join(qa_tuples_dir, f"para_{idx}_q_{qnum}.json")
                with open(file_path, "w") as file:
                    json.dump(good_qa_tuple, file, indent=4)

            vetted_qa_tuples.append(
                good_qa_tuple
            )  # We must filter out all None values at the end; but appending Nones lets us know where things went wrong, and how often.
    except Exception as e:
        print(f"Q ERROR: {e}")
        traceback.print_exc()



# Attempt to initialize filtered_worthy_for_questions
try:
    _ = filtered_worthy_for_questions
except NameError:
    filtered_worthy_for_questions = []

if not filtered_worthy_for_questions:
    # Load all files in the qa_tuples_dir if filtered_worthy_for_questions is not initialized
    existing_files = glob.glob(os.path.join(qa_tuples_dir, "*.json"))
    for file_path in existing_files:
        with open(file_path, "r") as file:
            qa_tuple = tuple(json.load(file))
        vetted_qa_tuples.append(qa_tuple)
else:
    tasks = [generate_qatuples_from_para(idx,para) for idx,para in enumerate(filtered_worthy_for_questions)]
    for future in asyncio.as_completed(tasks):
            await future
    

In [ ]:
print(
    "-------------- QUESTIONS CREATED ------------- STATS SO FAR (may be wrong if run was continued from interruption):"
)
nones = list(filter(lambda x: x[0] is None, vetted_qa_tuples))
print(f"Nones: {len(nones)}")
print(f"Non-nones: {len(vetted_qa_tuples) - len(nones)}")
print(f"Total: {len(vetted_qa_tuples)}")
# filter out all None values
vetted_qa_tuples = [qa for qa in vetted_qa_tuples if qa[0] is not None]
print("---------------- ONTO EXAMPLES GENERATION-------------------")

In [ ]:
# Check for and fix the common mistake: mentioning "the text".

writepath = "./qatuples_revised"


# Assuming vetted_qa_tuples is a list that might or might not exist
try:
    _ = vetted_qa_tuples
except NameError:
    vetted_qa_tuples = []


async def repair_qatuple_context(idx, tup):
    file_path = os.path.join(writepath, f"revised_{idx}.json")
    if os.path.exists(file_path):
        with open(file_path, "r", encoding="utf-8") as f:
            content = f.read()  # Read the file once and store its content
            print(file_path)
            if content == "failed":
                print("Loaded failed file")
                vetted_qa_tuples[idx] = None
                return None
            print("Loaded file:")
            print(content)
            try:
                data = json.loads(content)  # Convert the string back to JSON
                vetted_qa_tuples[idx] = (data[0], data[1], data[2], data[3])
                return None
            except json.JSONDecodeError:
                print("JSON decode error with the contents:", content)

    try:
        revision_id = make_id()
        revision, revision_output = await check_qatuple_context(tup, engine_wrapper)
        write_output_to_file(
            revision_output, "./question_context_revision_generations", revision_id
        )  # incidentally, identifying the problem and fixing it in the same step (without another planning step) works a lot better than identifying it and then trying to fix it in the next step.
        if isinstance(revision[0], str):  # if the thing was reworded
            vetted_qa_tuples[idx] = revision
        elif not revision[0]:
            vetted_qa_tuples[
                idx
            ] = None  # prepare item for deletion later; right now we just store it as None because indexes
        # else, if it passed, we just leave it be.

        # Write in-progress
        if not os.path.exists(writepath):
            os.makedirs(writepath)

        if vetted_qa_tuples[idx]:
            with open(file_path, "w") as file:
                json.dump(vetted_qa_tuples[idx], file, indent=4)
        else:
            with open(file_path, "w") as file:
                file.write("failed")

    except Exception as e:
        print("!!! ERROR!", e)
        traceback.print_exc()


# Load all files at the start if vetted_qa_tuples is empty
if not vetted_qa_tuples:
    # Check if the directory exists
    if os.path.exists(writepath):
        # List all files in directory
        for file_name in os.listdir(writepath):
            file_path = os.path.join(writepath, file_name)
            try:
                with open(file_path, "r", encoding="utf-8") as f:
                    content = f.read()
                    print(f"Loading file: {file_path}")
                    if content == "failed":
                        vetted_qa_tuples.append(None)
                    else:
                        try:
                            data = json.loads(content)
                            vetted_qa_tuples.append(
                                (data[0], data[1], data[2], data[3])
                            )
                        except json.JSONDecodeError:
                            print("JSON decode error with the contents:", content)
                            vetted_qa_tuples.append(None)
            except Exception as e:
                print(f"Error reading {file_path}: {e}")

else:
    old_tuples = vetted_qa_tuples.copy()
    
    tasks = [repair_qatuple_context(idx,tup) for idx, tup in enumerate(vetted_qa_tuples)]
    for future in asyncio.as_completed(tasks):
        await future

In [ ]:
# Print stats related to revised qatuples, and filter out nones (questions that were unanswerable due to lack of context).
import json
import os

print("-------------- QUESTIONS REVISED ------------- STATS SO FAR:")
nones = list(filter(lambda x: x is None, vetted_qa_tuples))
print(f"Nones: {len(nones)}")
print(f"Non-nones: {len(vetted_qa_tuples) - len(nones)}")
print(f"Total: {len(vetted_qa_tuples)}")
# filter out all None values
vetted_qa_tuples = [qa for qa in vetted_qa_tuples if qa is not None]
print("---------------- ONTO EXAMPLES GENERATION-------------------")

In [ ]:
# Group tuples for multiturn example generation (by chunk of source text) and then run that helper (so that we can make multiturn conversations from questions based on the same paragraphs)
def group_by_text(tuples_list):
    # Dictionary to hold the groups with text as the key
    groups = {}

    # Iterate over each tuple in the list
    for question, answer, text, textname in tuples_list:
        # If the text is not yet a key in the dictionary, add it with an empty list
        if text not in groups:
            groups[text] = []

        # Append the current tuple to the appropriate list
        groups[text].append((question, answer, text, textname))

    # Return the values of the dictionary, which are the lists of tuples grouped by text; also remove duplicates
    return [identify_duplicates(group) for group in list(groups.values())]

In [ ]:
qa_tuples_by_paragraph = group_by_text(vetted_qa_tuples)

In [ ]:
from math import ceil

# multiturn helpers
# These will probably be used for multiturn rapid-fire answering.


# Idea: use multiple short answers to train the task of answering multiple questions in one response. Two-three short answers per response should be enough.
async def make_multiturn_character(qa_tuples, conv_id):
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return "will_be_replaced", "will_be_replaced"
    plan, instructions, card_plan_output = await create_character_card_plan_many_tuples(
        qa_tuples, engine_wrapper
    )  # I will reuse the many tuples function for short question-answers, there's a lot of prompting in here already
    write_output_to_file(card_plan_output, "./multiturn_card_plan_generations", conv_id)
    char, char_output = await create_character_card_many_tuples(
        qa_tuples, plan, instructions, engine_wrapper
    )  # creates a character card
    write_output_to_file(char_output, "./multiturn_card_generations", conv_id)
    return char, instructions


async def make_multiturn_scenario(qa_tuples, character, conv_id):
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return "will_be_replaced", "will_be_replaced"
    plan, scenario_plan_output = await create_scenario_plan_many_tuples(
        qa_tuples, character, engine_wrapper
    )
    write_output_to_file(
        scenario_plan_output, "./multiturn_scenario_plan_generations", conv_id
    )
    scenario, scenario_output = await create_scenario_many_tuples(
        qa_tuples, character, plan, engine_wrapper
    )  # creates a scenario based on a character card and question/answer tuple
    write_output_to_file(scenario_output, "./multiturn_scenario_generations", conv_id)
    return scenario, plan


async def make_multiturn_conversation_info(qa_tuples):
    conv_id = make_id()
    if (
        ASSISTANT_MODE
    ):  # If assistant mode is on, multiturn convs will have hardcoded information in its prompt file; but we still need to put something in the file
        return (qa_tuples, "will", "be", "replaced", conv_id)
    # thought_plan = create_thought_plan_many_tuples(qa_tuples,character,scenario,logic_llm) # There IS a way to make multiturn chain of thought answering work: generate each pair of messages using a separate prompt or a separate function, each of which has only the thought plan for that question/answer pair. But simply cramming in all the step-by-step things will confuse the hell out of the poor model. So for the first release version we're skipping it and just giving the response, with no reasoning, in the multiturn convs.
    character, instructions = await make_multiturn_character(qa_tuples, conv_id)
    scenario, scenario_plan = await make_multiturn_scenario(qa_tuples, character, conv_id)

    return (qa_tuples, character, scenario, scenario_plan, conv_id)

In [ ]:
import os
import json
import random
import itertools


if not os.path.exists(multi_turn_convs_info_dir):
    os.makedirs(multi_turn_convs_info_dir)

In [ ]:
multi_turn_convs_info = []

async def create_info(idx, group):
    all_permutations = list(itertools.permutations(group))

    sample_size = min(REARRANGEMENTS_TO_TAKE, len(all_permutations))
    sampled_permutations = random.sample(all_permutations, sample_size)

    group_convs_info = []

    for iter, perm in enumerate(sampled_permutations):
        file_path = os.path.join(multi_turn_convs_info_dir, f"info_{idx}_{iter}.json")

        # Skip if file already exists
        if not os.path.exists(file_path):
            try:
                info = await make_multiturn_conversation_info(perm)

                if info is not None:
                    with open(file_path, "w") as file:
                        json.dump(info, file, indent=4)

                group_convs_info.append(info)
            except Exception as e:
                print("ERROR!!!!--!!!!", e)
        else:
            print(f"Skipped generating {file_path} as it already exists")

    multi_turn_convs_info.append(group_convs_info)

tasks = [create_info(idx,group) for idx,group in enumerate(qa_tuples_by_paragraph)]
for future in asyncio.as_completed(tasks):
    await future

In [ ]:
del engine_wrapper  # Maybe this frees the engine? IDK

# Stop Here, Restart the Notebook, and Reimport Everything IF you are doing 2-step Generation (where you do the easy bits with a small model, and the hard bit with a large one).

In [ ]:
engine_wrapper = EngineWrapper(model=LARGE_LOGICAL_MODEL, quantization="gptq")

In [ ]:
import os
import json


def read_json_files_info(directory):
    # Create a list to hold the tuples
    tuple_list = []

    # Get all the .json files in the directory, sorted
    json_files = sorted([f for f in os.listdir(directory) if f.endswith(".json")])

    # Read each file and convert the contents
    for file in json_files:
        with open(os.path.join(directory, file), "r") as f:
            data = json.load(f)
            # Ensure the data is in the correct format before converting to tuple
            if (
                isinstance(data, list)
                and len(data) == 5
                and isinstance(data[0], list)
                and all(len(item) == 4 for item in data[0])
                and all(isinstance(i, str) for i in data[1:])
            ):
                tuple_list.append((data[0], data[1], data[2], data[3], data[4]))

    return tuple_list


convs_info = read_json_files_info(multi_turn_convs_info_dir)

In [ ]:
import os
import json
import random
import itertools
import asyncio

multi_turn_convs_dir = "./multi_turn_convs"
if not os.path.exists(multi_turn_convs_dir):
    os.makedirs(multi_turn_convs_dir)

multi_turn_convs = []
async def create_conversation(idx,info):
    file_path = os.path.join(multi_turn_convs_dir, f"conv_{idx}.json")

    # Skip if file already exists
    if not os.path.exists(file_path):
        try:
            conv = await make_multiturn_conversation(info, engine_wrapper)
            final_conv = await control_flow_functions.ensure_multiple_answers_are_same(info, conv, engine_wrapper)

            if final_conv is not None:
                with open(file_path, "w") as file:
                    json.dump(final_conv, file, indent=4)

            multi_turn_convs.append(final_conv)
        except Exception as e:
            print("Had an error, retrying...", e)
    else:
        with open(file_path, "r", encoding="utf-8") as f:
            data = json.load(f)
            multi_turn_convs.append(data)
        print(f"Skipped generating {file_path} as it already exists")
        
tasks = [create_conversation(idx,info) for idx,info in enumerate(convs_info)]
for future in asyncio.as_completed(tasks):
    await future

# Yay! Now you have a dataset!
### GPT wrote the cell below. I think it successfully converts things to ShareGPT format for use with axolotl, but I am not sure because I don't know that format very well and haven't used Axolotl. However, the json produced by the second function looks fine.

In [ ]:
import os
import json


def convert_directory_to_list(directory_path):
    master_list = []
    simplified_list = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            filepath = os.path.join(directory_path, filename)
            with open(filepath, "r") as file:
                data = json.load(file)
                if isinstance(data, list) and all(
                    isinstance(item, (list, str)) for item in data
                ):
                    master_list.append(data)

                    # Extract and process conversation
                    conversation, primary_char_desc = data[0], data[1]
                    primary_char_name = extract_name(primary_char_desc)
                    dialogues = extract_conversation(conversation)

                    # Convert to simplified format
                    simplified_conversations = []
                    for i, (charname, message) in enumerate(
                        dialogues
                    ):  # Skipping the first message
                        from_person = (
                            "human" if charname == primary_char_name else "gpt"
                        )
                        simplified_conversations.append(
                            {"from": from_person, "value": f"{charname}: {message}"}
                        )

                    if simplified_conversations:  # If there are any conversations
                        simplified_list.append(
                            {"conversations": simplified_conversations}
                        )

    # Write the master list to a new .jsonl file
    with open("master_list.jsonl", "w") as file:
        for item in master_list:
            file.write(json.dumps(item) + "\n")

    # Write the simplified data to a different .jsonl file
    with open("simplified_data.jsonl", "w") as file:
        for item in simplified_list:
            file.write(json.dumps(item) + "\n")

    print(
        "Conversion complete. Master list written to 'master_list.json'. Simplified data written to 'simplified_data.json'."
    )


convert_directory_to_list("./multi_turn_convs/")

# Write version where each line in the conversation is separate, so that you do not need to write an extract_convs function if you want a custom format.


def convert_directory_and_process_conversations(directory_path):
    master_list = []

    for filename in os.listdir(directory_path):
        if filename.endswith(".json"):
            filepath = os.path.join(directory_path, filename)
            with open(filepath, "r") as file:
                data = json.load(file)

                if isinstance(data, list) and all(
                    isinstance(item, (list, str)) for item in data
                ):
                    # Extract and process the conversation part
                    conversations = extract_conversation(data[0])
                    # Convert tuples back to the formatted string as required
                    data[0] = [
                        f"{charname}: {message}" for charname, message in conversations
                    ]
                    master_list.append(data)
                else:
                    print(f"File {filename} is not in the expected format.")

    # Write the master list to a new file
    with open("processed_master_list.json", "w") as file:
        json.dump(master_list, file)

    print(
        "Conversion complete. The processed master list is written to 'processed_master_list.json'."
    )


# Usage: replace 'path_to_directory' with the actual directory path
convert_directory_and_process_conversations("./multi_turn_convs/")

In [ ]:
with open("./processed_master_list.json") as f:
    first = f.read()
    data = json.loads(first)

In [ ]:
# For curiosity's sake, you can find out how many lines of dialogue you generated
def filter_and_flatten(lst):
    # Initialize an empty list to hold the flattened elements
    flat_list = []

    # Loop through each sublist in the main list
    for sublst in lst:
        # Check if the first element of the sublist is itself a list (subsublist1)
        if isinstance(sublst[0], list):
            # Extend the flat_list with the elements from subsublist1
            flat_list.extend(sublst[0])

    return flat_list


len(filter_and_flatten(data))